In [1]:
import pandas as pd

import pandas as pd

In [2]:
#Imporing the data and transforming it into useable dataset

In [3]:
dataBin = pd.read_csv('order_brush_order.csv')

In [4]:
dataBin = dataBin.drop(['orderid'], axis='columns')

In [5]:
from datetime import datetime

In [6]:
dFormat = '%Y-%m-%d %H:%M:%S'
dataBin['year'] = dataBin['event_time'].apply(lambda x: int(datetime.strptime(x, dFormat).strftime("%Y")))
dataBin['month'] = dataBin['event_time'].apply(lambda x:  int(datetime.strptime(x, dFormat).strftime("%m")))
dataBin['date'] = dataBin['event_time'].apply(lambda x: int( datetime.strptime(x, dFormat).strftime("%d")))
dataBin['hour'] = dataBin['event_time'].apply(lambda x: int( datetime.strptime(x, dFormat).strftime("%H")))
dataBin['minute'] = dataBin['event_time'].apply(lambda x: int( datetime.strptime(x, dFormat).strftime("%M")))

In [7]:
dataBin = dataBin.drop(['event_time'], axis='columns')
dataBin = dataBin.drop(['month'], axis = 'columns')
dataBin = dataBin.drop(['year'], axis = 'columns')

In [8]:
# Applying data transformation for analysis

In [9]:
dataBin['timePoint'] = ((dataBin['date'] - 27) * 60*24) +  dataBin['hour'] * 60 + dataBin['minute']

In [10]:
transactCt = dataBin.groupby('shopid')['shopid'].agg('count')

In [11]:
listShop = dataBin['shopid'].unique()

In [12]:
frequentShops = dict()
for items in listShop:
    if (transactCt[int(items)] >= 1):
        frequentShops[items] = transactCt[int(items)]

In [13]:
# Setting up functions for data analysis

In [14]:
def checkBrushing():
    shopList = dict()
    
    for shops in frequentShops:
        
        shopList[shops] = list()
        
        x = dataBin[dataBin['shopid'] == shops]
        x = x.sort_values(by = ['timePoint'])
        
        
        brush_oc = 0
        ctr_time = -999
        pend_list = list()
        x_handler = None
        
        for ctr in x['timePoint'].index:
            if (ctr_time == -999):
                ctr_time = x['timePoint'][ctr]
                x_handler = x['userid'][ctr]
                
            elif (abs(x['timePoint'][ctr] - ctr_time) > 10):
                
                ctr_time = x['timePoint'][ctr]
                x_handler = x['userid'][ctr]
                #print(ctr_time)
                
                if (brush_oc >= 2):
                    #print(pend_list)
                    for pending in pend_list:
                        shopList[shops].append(pending)
                
                brush_oc = 0
                pend_list = list()
                
            else:
                brush_oc += 1

                if (brush_oc == 1):
                    pend_list.append(x_handler)
                    pend_list.append(x['userid'][ctr])
                elif (brush_oc > 1):
                    pend_list.append(x['userid'][ctr])
                    
                    
                ctr_time = x['timePoint'][ctr]
                    
    return shopList                

In [15]:
def occurence_counter(List):
    retList = dict()
    
    for item in List:
        if item in retList:
            retList[item] += 1
        else:
            retList[item] = 1
    
    returning = list()
    
    for key in retList:
        if (retList[key] >= 2):
            returning.append(key)
            
    return returning
    
    

In [16]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    if (len(s) != 0):
        for ele in s:  
            str1 += str(ele)
            str1 += "&"
        
        str1 = str1[:len(str1)-1]
    
    # return string   
    return str1  

In [17]:
# Data Analysis

In [18]:
extract = checkBrushing()

In [19]:
dataBin2 = pd.DataFrame()
dataBin2['shopid'] = [i for i in extract]

In [20]:
dataBin2['count'] = [len(extract[i]) for i in extract]

In [21]:
dataBin2['userid'] = [occurence_counter(extract[i]) for i in extract]

In [22]:
#Removal of filler buyers

In [23]:
dataBin2 = dataBin2[dataBin2['count'] > 3]

In [24]:
dataBin2 = dataBin2[dataBin2['userid'] != 0]

In [25]:
dataBin2['userid2'] = dataBin2['userid'].apply(lambda x: listToString(x))

In [26]:
dataBin3 = dataBin2[['shopid','userid2']]

In [27]:
dataBin3 = dataBin3.rename(columns = {'userid2':'userid'})

In [28]:
dataBinFinal = pd.DataFrame()

In [29]:
dataBinFinal['shopid'] = listShop

In [30]:
dataBinFinal = pd.merge(dataBinFinal, dataBin3, how='left')

In [31]:
dataBinFinal['userid'] = dataBinFinal['userid'].apply(lambda x: '0' if ((x == 0) or (x == '')) else x)

In [32]:
dataBinFinal = dataBinFinal.fillna(0)

In [33]:
dataBinFinal.to_csv('exprtCSV.csv', index=False)

In [34]:
dataBinFinal.groupby('userid')['userid'].agg('count')

userid
0                                                          18071
0                                                            247
100793597&55375223&123073475&14512957&97325553&96470129        1
10209247                                                       1
102125116                                                      1
                                                           ...  
9837180                                                        1
9837180&37720791&123365889                                     1
98771800&159224660&131375087                                   1
99177476                                                       1
99691479                                                       1
Name: userid, Length: 451, dtype: int64

In [35]:
#dataBin2.sort_values(by = ['count'], ascending=False)

In [36]:
dataBinFinal.shape

(18770, 2)

In [37]:
dataBin3.sort_values(by = ['userid'], ascending=False)

,shopid,userid
2279,149773657,99691479
2137,11342466,99177476
1203,51960315,98771800&159224660&131375087
2624,54054704,9837180&37720791&123365889
1011,49674796,9837180
...,...,...
720,50090018,
2204,90172857,
2189,10678993,
736,99510463,


In [38]:
dataBinFinal[dataBinFinal['userid'] != '0']

,shopid,userid
1,156423439,0
3,63674025,0
4,127249066,0
5,173811070,0
6,107921853,113289032&4950388&101766677&78896980&2954376&2...
...,...,...
18765,163337224,0
18766,772599,0
18767,50236030,0
18768,203587596,0


In [39]:
len(frequentShops)

18770

In [40]:
dataBin.describe()

,shopid,userid,date,hour,minute,timePoint
count,2.227500e+05,2.227500e+05,222750.000000,222750.000000,222750.000000,222750.000000
mean,9.433117e+07,9.802880e+07,29.025917,13.401715,29.195232,3750.618532
std,5.695790e+07,6.839048e+07,1.393320,6.929045,17.393935,2037.957849
min,1.000900e+04,1.000700e+04,27.000000,0.000000,0.000000,0.000000
25%,4.980267e+07,3.508127e+07,28.000000,9.000000,14.000000,2139.000000
50%,9.033636e+07,9.309625e+07,29.000000,14.000000,29.000000,3840.000000
75%,1.475053e+08,1.590612e+08,30.000000,19.000000,44.000000,5513.000000
max,2.154352e+08,2.155262e+08,31.000000,23.000000,59.000000,7199.000000
